# Using GraphRAG-SDK to Create a Multi-Agent-Orchestrator RAG System based on Knowledge Graph from Structured Data

GraphRAG-SDK provides a powerful tool, enhanced by LLM technology, for building a Multi-Agent-Orchestrator RAG System. This example demonstrates how to create a knowledge graph-based multi-agent system from structured data and how to use the orchestrator to manage the agents and perform question answering.

In [ ]:
!pip install graphrag_sdk

In [1]:
from json import loads
from dotenv import load_dotenv
from graphrag_sdk.orchestrator import Orchestrator
from graphrag_sdk.agents.kg_agent import KGAgent
from graphrag_sdk.models.openai import OpenAiGenerativeModel
from graphrag_sdk import (
    Ontology, Entity, Relation, Attribute, AttributeType, KnowledgeGraph, KnowledgeGraphModelConfig
)

# Load environment variables
load_dotenv()

# Configuration
# OPENAI_API_KEY = "sk-"# OpenAI API key

True

### Ontology - Predefined Ontology

We will begin with an initial ontology for the agents, based on the knowledge that interests us.


In [2]:
# Restaurants Ontology
restaurants_ontology = Ontology()

# Attractions Ontology
attractions_ontology = Ontology()

# Manually created Ontology by adding entities and relations
restaurants_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)

attractions_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)

### KG from Structure Data

Creating the KG from the data that will be used by the agents

In [3]:
# Define the model
model = OpenAiGenerativeModel("gpt-4o")

# Create the KG from the predefined ontology.
# Restaurants KG
restaurants_kg = KnowledgeGraph(
    name="restaurants",
    ontology=restaurants_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Attractions KG
attractions_kg = KnowledgeGraph(
    name="attractions",
    ontology=attractions_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Add nodes and edges to the KGs from the data
with open("examples/trip/data/cities.json") as f:
    cities = loads(f.read())
with open("examples/trip/data/restaurants.json") as f:
    restaurants = loads(f.read())
with open("examples/trip/data/attractions.json") as f:
    attractions = loads(f.read())

for city in cities:
    restaurants_kg.add_node(
        "City",
        {
            "name": city["name"],
            "weather": city["weather"],
            "population": city["population"],
        },
    )
    restaurants_kg.add_node("Country", {"name": city["country"]})
    restaurants_kg.add_edge(
        "IN_COUNTRY",
        "City",
        "Country",
        {"name": city["name"]},
        {"name": city["country"]},
    )

    attractions_kg.add_node(
        "City",
        {
            "name": city["name"],
            "weather": city["weather"],
            "population": city["population"],
        },
    )
    attractions_kg.add_node("Country", {"name": city["country"]})
    attractions_kg.add_edge(
        "IN_COUNTRY",
        "City",
        "Country",
        {"name": city["name"]},
        {"name": city["country"]},
    )

for restaurant in restaurants:
    restaurants_kg.add_node(
        "Restaurant",
        {
            "name": restaurant["name"],
            "description": restaurant["description"],
            "rating": restaurant["rating"],
            "food_type": restaurant["food_type"],
        },
    )
    restaurants_kg.add_edge(
        "IN_CITY",
        "Restaurant",
        "City",
        {"name": restaurant["name"]},
        {"name": restaurant["city"]},
    )

for attraction in attractions:
    attractions_kg.add_node(
        "Attraction",
        {
            "name": attraction["name"],
            "description": attraction["description"],
            "type": attraction["type"],
        },
    )
    attractions_kg.add_edge(
        "IN_CITY",
        "Attraction",
        "City",
        {"name": attraction["name"]},
        {"name": attraction["city"]},
    )

### Agents

Creating the Agents based on the KG.

In [4]:
# The following agent is specialized in finding restaurants.
restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="I'm a restaurant agent, specialized in finding the best restaurants for you.",
)

# The following agent is specialized in finding tourist attractions.
attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="I'm an attractions agent, specialized in finding the best attractions for you.",
)

### Orchestrator - Multi-Agent System

The orchestrator manages the usage of agents and handles questioning. Each agent is registered with the orchestrator, which determines how to engage the agents to provide the best possible answer when a question is asked.

In [5]:
# Initialize the orchestrator and register agents
orchestrator = Orchestrator(
    model,
    backstory="You are a trip planner, and you want to provide the best possible itinerary for your clients.",
)
orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

# Query the orchestrator
runner = orchestrator.ask("Create a two-day itinerary for a trip to Rome. Please don't ask me any questions. Just provide the best itinerary you can.")

In [6]:
print(runner.output)

Certainly! Here's a detailed two-day itinerary for your trip to Rome, ensuring you make the most out of your visit to this historically rich and vibrant city. We’ve arranged a balanced mix of iconic attractions and delightful dining experiences for your enjoyment.

**Day 1: Discovering Ancient Rome and Modern Flavors**
- **Morning: Colosseum**
  Start your day by visiting the Colosseum, the iconic ancient amphitheater known for its gladiatorial contests and grand spectacles. Allow yourself a few hours to explore the structure and absorb the rich history of ancient Rome that it represents.

- **Lunch: Tonnarello**
  After exploring, head to Tonnarello, a restaurant praised for its casual ambiance and delicious Roman cuisine. Enjoy dishes like homemade pasta and meatballs while soaking in the atmosphere of authentic Roman hospitality. 

- **Afternoon: Trevi Fountain**
  Rejuvenated from lunch, stroll towards the Trevi Fountain. This Baroque masterpiece not only offers stunning sculptures

In [7]:
runner = orchestrator.ask("Please tell me only the name of the restaurant for dinner at the first day that you mention in the itinerary of the trip")
print(runner.output)

Certainly! Based on the two-day itinerary for Rome that was previously provided to you, the restaurant mentioned for dinner on the first day is "Il Pagliaccio." This restaurant is renowned for its elegant ambiance and Michelin-starred contemporary Italian cuisine, offering a perfect dining experience to conclude your day of exploring ancient Rome. Enjoy your dinner and the remarkable culinary delights it has to offer!


In [8]:
runner = orchestrator.ask("Can you change this restaurant to another one and give me the updated itinerary?")
print(runner.output)

Certainly! I've updated your two-day itinerary for Rome with a new dinner option for the first day. Here is the revised itinerary:

**Day 1: Discovering Ancient Rome and Modern Flavors**
- **Morning: Colosseum**
  Begin your day with a visit to the iconic Colosseum, where you'll delve into the historical essence of ancient Rome. Take a few hours to explore this magnificent structure and the stories it holds about the past.

- **Lunch: Tonnarello**
  Enjoy your lunch at Tonnarello, a restaurant highly praised for its casual ambiance and delicious Roman cuisines such as pasta and meatballs. This spot also features patio seating and acoustic guitar, enhancing your dining experience.

- **Afternoon: Trevi Fountain**
  After lunch, take a leisurely walk to the Trevi Fountain. Revel in the beauty of this Baroque masterpiece, famous for its stunning sculptures and the tradition of tossing a coin for a promise to return.

- **Dinner: Tonnarello (Alternative choice)**
  For dinner, continue enj

In [9]:
runner = orchestrator.ask("Please tell me attraction on the morning at the first day that you mention in the itinerary of the trip")
print(runner.output)

Certainly! For the two-day itinerary in Rome that was previously provided, the attraction mentioned for the morning on the first day is the **Colosseum**. This iconic ancient amphitheater is known for its gladiatorial contests and grand spectacles, making it a must-see when exploring the history and culture of ancient Rome. Visiting the Colosseum allows you to delve into Rome's fascinating past and appreciate the architectural marvels and historical significance of this world-renowned site. Enjoy your exploration of this magnificent landmark!
